In [ ]:
import os
import scipy
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
ODIR = 'out'
FFMT = 'png'
FDPI = 200

# Configure rcParams
size  = {'major': 6, 'minor': 3}
width = {'major': 1, 'minor': 1}
for xy in ['xtick', 'ytick']:
    for t in ['major', 'minor']:
        mpl.rcParams[f'{xy}.{t}.size'] = size[t]
        mpl.rcParams[f'{xy}.{t}.width'] = width[t]
    mpl.rcParams[f'{xy}.direction'] = 'in'
    mpl.rcParams[f'{xy}.color'] = 'white'
mpl.rcParams['text.usetex'] = True

## Fetch and build data from AAVSO observations

Similarly as https://github.com/hippke/betelbot/ does it.

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# Iterate over the first N pages of Alpha Ori (Betelgeuse) observations
# and gather all Betelgeuse data from them
jd = []
mg = []

url_base = 'https://apps.aavso.org/webobs/results/?star=000-BBW-825&num_results=200&obs_types=vis&page={}'
for p_i in tqdm(range(1, 60)):
    # Fetch and parse the HTML5 page
    r = requests.get(url_base.format(p_i))
    soup = BeautifulSoup(markup=r.content, features='html.parser')
    # Rows of data are contained in the <table> element with a <thead> element
    # containing a header and a <tbody> element containing the data
    #
    # The first and last column of these headers are just bad HTML, drop them
    header = soup.select_one('thead').select('th')[1:-1]
    # The rows are contained in <tr> elements inside a singular <tbody> element
    table = soup.select('tbody tr')
    # Relevant data in each row are stored in every 4th <tr> tag, which we
    # should extract the Julian Date (3th column) and magnitude (5th column) from
    for row in table[::4]:
        jd.append(row.select('td')[2].get_text())
        mg.append(row.select('td')[4].get_text().lstrip('<>'))
jd = np.array(jd, dtype=np.float64)
mg = np.array(mg, dtype=np.float64)

## Convert Julian Date to Gregorian Date

In [ ]:
def jd2g(jd):
    '''Convert Julian Date to Gregorian Datetime in a NumPy and Pandas
    compatible format.

    Algorithm is the same as in `jdcal.jd2gcal()` seen in
    https://github.com/phn/jdcal/blob/master/jdcal.py#L193.

    Parameters
    ----------
    jd : scalar or array_like
        Julian Date or array of Julian Dates.
    '''
    f, jd_i = np.modf(jd)

    # Calculate years, months and days
    ell = jd_i + 68569
    n = np.int64((4 * ell) / 146097.0)
    ell -= np.int64(((146097 * n) + 3) / 4.0)
    i = np.int64((4000 * (ell + 1)) / 1461001)
    ell -= np.int64((1461 * i) / 4.0) - 31
    j = np.int64((80 * ell) / 2447.0)
    D = ell - np.int64((2447 * j) / 80.0)
    ell = np.int64(j / 11.0)
    M = j + 2 - (12 * ell)
    Y = 100 * (n - 49) + i + ell

    # Calculate hours, minutes and seconds
    r, HH = np.modf(f*24)
    r, MM = np.modf(r*60)
    r, SS = np.modf(r*60)

    T = np.stack((Y, M, D, HH, MM, SS), axis=-1)
    T = np.array(T, copy=False, dtype=np.int64)

    # Convert values to integers and then to `numpy.datetime64` arrays
    fmt = '{:04}-{:02}-{:02}T{:02}:{:02}:{:02}'
    gd = pd.DatetimeIndex([
        fmt.format(y, m, d, hh, mm, ss) for y, m, d, hh, mm, ss in T
    ])

    return gd

In [ ]:
gd = jd2g(jd)
jd_i = np.int64(jd)
gd_i = jd2g(jd_i)
ujd = np.unique(jd_i)
ugd = jd2g(ujd)

## Aggregate magnitudes and errors

In [ ]:
df = pd.DataFrame(data={'jd': jd, 'gd': gd, 'mag': mg})
group = df['mag'].groupby(jd_i)
df_agg = pd.DataFrame(data={
        'gd': ugd,
        'mean': group.mean(),
        'std': group.std()
})

In [ ]:
# Drop observations outside a specified time interval
min_date = pd.Timestamp('2021-08-01')
max_date = pd.Timestamp('2024-09-01')
df_agg = df_agg[(df_agg['gd'] > min_date) & (df_agg['gd'] < max_date)]

# Drop outlier observations
max_std = 3
df_mean, df_std = df_agg['mean'].mean(), df_agg['mean'].std()
df_agg = df_agg[abs(df_agg['mean'] - df_mean) < max_std*df_std]

# Exclude days, where only a single observation was made
# (or where a single observation remained after the filtering above)
df_agg.dropna(axis=0, how='any', inplace=True)

## Plot data

In [ ]:
def set_colors(arr, cmap,
               *,
               vmin=0.0, vmax=1.0, imin=0.0, imax=1.0):
    '''Create and scale a colormap for an interval of the values in an
    array, using SciPy's 1D interpolation.

    Parameters
    ----------
    arr : array-like
        Input values to scale the colormap to.
    cmap : 
        Any matplotlib colormap the 
    '''

    arr_c = np.array(arr, copy=True)
    amin, amax = np.min(arr_c), np.max(arr_c)

    mmin = amin + imin*(amax - amin)
    mmax = amin + imax*(amax - amin)
    arr_c[arr_c < mmin] = mmin
    arr_c[arr_c > mmax] = mmax

    # Scale the colorscale with
    m = scipy.interpolate.interp1d(
        x=[mmax, mmin], y=[vmin, vmax], kind='linear'
    )

    c = cmap(m(arr_c))

    return c

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=140,
                       facecolor='black', subplot_kw=dict(facecolor='black'))
for spine in ax.spines.values():
    spine.set_edgecolor('white')

colors = set_colors(df_agg['mean'], cmap=cm.magma,
                    vmin=0.5, vmax=1.0, imin=0.0, imax=1.0)
ax.scatter(df_agg['gd'], df_agg['mean'],
           c=colors, s=2**3, alpha=0.8, zorder=1)
ax.errorbar(df_agg['gd'], df_agg['mean'], yerr=df_agg['std'],
            ecolor=colors, elinewidth=0.5, linestyle='None',
            alpha=0.2, zorder=0)

ax.set_xlim(None, df_agg['gd'].values[-1] + pd.Timedelta(1, 'W'))
ax.set_ylim(9, 11)
ax.tick_params(axis='both', which='both', labelcolor='white')
ax.tick_params(axis='x', which='both', labelrotation=30)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))

ax.invert_yaxis()

ax.set_ylabel('\\textbf{Magnitude}',
              color='white', fontsize=24)

ax.set_title('\\textbf{Fig. 1.} AAVSO light curve of \\textit{T Coronae Borealis}', y=-0.2,
             fontsize=18, color='white')

# Pre-eruption dimming?
ax.text(x=np.datetime64('2023-10-15'), y=10.85, s='\\texttt{Pre-eruption dimming?}',
        color='white', fontsize=12, alpha=0.8,
        ha='center', va='bottom',
        bbox=dict(facecolor='black', alpha=0.2, lw=0))

# Source text
ax.text(x=0.99, y=0.975, s='\\texttt{Source of data: https://www.aavso.org/}',
        color='white', fontsize=8, alpha=0.8,
        ha='right', va='top', transform=ax.transAxes,
        bbox=dict(facecolor='black', alpha=0.2, lw=0))

# Credit
ax.text(x=0.01, y=0.02, s='\\texttt{@masterdesky}',
        color='white', fontsize=8, alpha=0.4,
        ha='left', va='bottom', transform=ax.transAxes,
        bbox=dict(facecolor='black', alpha=0.2, lw=0))

os.makedirs(ODIR, exist_ok=True)
fig.savefig(os.path.join(ODIR, f'TCrB.{FFMT}'), format=FFMT, dpi='figure',
            bbox_inches='tight', pad_inches='layout',
            facecolor='auto', edgecolor='auto', backend=None)
plt.show()